# Data Understanding


## Preperation

Import packages and set globals


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option("display.width", 1000)
pd.set_option("display.max_columns", None)
pd.options.display.float_format = "{:.2f}".format

%matplotlib inline
plt.rcParams["figure.figsize"] = (20, 6)

In [2]:
gdp_df = pd.read_pickle("../data/raw/gdp.pkl")
occ_df = pd.read_pickle("../data/raw/naics_occupation.pkl")
ptn_df = pd.read_pickle("../data/raw/naics_pattern.pkl")

Based on instruction given to us, we preemptivly drop uninteresting NAICS


In [3]:
naics_filter = "|".join(["^11", "^21", "^22", "^23", "^31", "^32", "^33"])

gdp_df = gdp_df.loc[gdp_df["IndustryClassification"].str.contains(naics_filter)]
occ_df = occ_df.loc[occ_df["naics"].str.contains(naics_filter)]
ptn_df = ptn_df.loc[ptn_df["naics"].str.contains(naics_filter)]

## Structure

Brief overview of the different datasets given to us


### Data Format


In [4]:
gdp_df.head(10)

,FIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2017,2018,2019,2020,2021,2022
2,0,United States,,CAGDP9,3,11,"Agriculture, forestry, fishing and hunting",Thousands of chained 2017 dollars,176840000.00,184105000.00,171149000.00,173659000.00,183742000.00,170058000.00
3,0,United States,,CAGDP9,6,21,"Mining, quarrying, and oil and gas extraction",Thousands of chained 2017 dollars,267302000.00,277013000.00,314604000.00,305588000.00,269478000.00,245290000.00
4,0,United States,,CAGDP9,10,22,Utilities,Thousands of chained 2017 dollars,313711000.00,309269000.00,312920000.00,332336000.00,316603000.00,318392000.00
5,0,United States,,CAGDP9,11,23,Construction,Thousands of chained 2017 dollars,840220000.00,863755000.00,882046000.00,856487000.00,888104000.00,827768000.00
6,0,United States,,CAGDP9,12,31-33,Manufacturing,Thousands of chained 2017 dollars,2109718000.00,2213031000.00,2223326000.00,2127060000.00,2248052000.00,2277757000.00
7,0,United States,,CAGDP9,13,"321,327-339",Durable goods manufacturing,Thousands of chained 2017 dollars,1178271000.00,1227629000.00,1241367000.00,1171720000.00,1249509000.00,1296970000.00
8,0,United States,,CAGDP9,25,"311-316,322-326",Nondurable goods manufacturing,Thousands of chained 2017 dollars,931447000.00,985229000.00,981959000.00,955683000.00,999648000.00,985406000.00
28,0,United States,,CAGDP9,87,"11,21",Natural resources and mining,Thousands of chained 2017 dollars,444143000.00,460949703.00,487685301.00,483389355.00,455053638.00,416036526.00
30,0,United States,,CAGDP9,89,"22,48-49",Transportation and utilities,Thousands of chained 2017 dollars,949174000.00,966855752.00,971436131.00,936020785.00,1000941475.00,1016052876.00
31,0,United States,,CAGDP9,90,"31-33,51",Manufacturing and information,Thousands of chained 2017 dollars,3119738000.00,3278878278.00,3396542443.00,3342047889.00,3639162084.00,3758807082.00


In [5]:
occ_df.head(10)

,FIPS,State_GEOID,naics,NAICS_TITLE,emp_total_county_naics,OCC_CODE,OCC_TITLE,emp_occupation
0,13073,13,2373,"Highway, Street, and Bridge Construction",27,11-3051,Industrial Production Managers,0.02
1,13073,13,2381,"Foundation, Structure, and Building Exterior C...",231,11-3051,Industrial Production Managers,0.07
2,13073,13,2382,Building Equipment Contractors,868,11-3051,Industrial Production Managers,0.13
3,13073,13,2383,Building Finishing Contractors,281,11-3051,Industrial Production Managers,0.08
5,13073,13,3211,Sawmills and Wood Preservation,116,11-3051,Industrial Production Managers,0.98
6,13073,13,3219,Other Wood Product Manufacturing,27,11-3051,Industrial Production Managers,0.26
7,13073,13,3231,Printing and Related Support Activities,24,11-3051,Industrial Production Managers,0.26
8,13073,13,3327,"Machine Shops; Turned Product; and Screw, Nut,...",18,11-3051,Industrial Production Managers,0.31
9,13073,13,3363,Motor Vehicle Parts Manufacturing,58,11-3051,Industrial Production Managers,0.93
10,13073,13,3370A1,"3371, 3372",108,11-3051,Industrial Production Managers,0.97


In [6]:
ptn_df.head(10)

,State_GEOID,County_GEOID,FIPS,naics_2,naics,DESCRIPTION,emp_nf,emp,qp1_nf,qp1,ap_nf,ap,est,n<5,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,n1000_1,n1000_2,n1000_3,n1000_4
0,01,001,1001,11,1133,Logging,G,68,G,1213,G,4563,7,6,N,N,N,N,N,N,N,N,N,N,N,N
1,01,001,1001,21,2123,Nonmetallic Mineral Mining and Quarrying,G,87,G,1224,G,5144,6,N,N,N,N,N,N,N,N,N,N,N,N,N
2,01,001,1001,22,2211,"Electric Power Generation, Transmission and Di...",G,129,G,4418,G,16342,4,N,N,N,N,N,N,N,N,N,N,N,N,N
4,01,001,1001,23,2362,Nonresidential Building Construction,H,69,H,685,H,4184,7,3,N,N,N,N,N,N,N,N,N,N,N,N
5,01,001,1001,23,2371,Utility System Construction,H,65,H,1117,H,4574,4,N,N,N,N,N,N,N,N,N,N,N,N,N
6,01,001,1001,23,2381,"Foundation, Structure, and Building Exterior C...",H,44,G,401,G,1775,12,7,4,N,N,N,N,N,N,N,N,N,N,N
7,01,001,1001,23,2382,Building Equipment Contractors,H,157,G,1514,G,7822,27,15,7,4,N,N,N,N,N,N,N,N,N,N
8,01,001,1001,23,2383,Building Finishing Contractors,H,81,H,753,H,3630,13,7,5,N,N,N,N,N,N,N,N,N,N,N
10,01,001,1001,32,3231,Printing and Related Support Activities,J,33,H,280,H,1334,4,N,N,N,N,N,N,N,N,N,N,N,N,N
54,01,003,1003,21,2123,Nonmetallic Mineral Mining and Quarrying,J,29,J,306,J,1291,3,N,N,N,N,N,N,N,N,N,N,N,N,N


### Info


In [7]:
gdp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31780 entries, 2 to 108049
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   FIPS                    31780 non-null  int64  
 1   GeoName                 31780 non-null  object 
 2   Region                  31780 non-null  object 
 3   TableName               31780 non-null  object 
 4   LineCode                31780 non-null  int64  
 5   IndustryClassification  31780 non-null  object 
 6   Description             31780 non-null  object 
 7   Unit                    31780 non-null  object 
 8   2017                    26563 non-null  float64
 9   2018                    26499 non-null  float64
 10  2019                    26259 non-null  float64
 11  2020                    26472 non-null  float64
 12  2021                    26580 non-null  float64
 13  2022                    27125 non-null  float64
dtypes: float64(6), int64(2), object(6)
memory 

In [8]:
occ_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 711429 entries, 0 to 1820150
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   FIPS                    711429 non-null  int64  
 1   State_GEOID             711429 non-null  int64  
 2   naics                   711429 non-null  object 
 3   NAICS_TITLE             711429 non-null  object 
 4   emp_total_county_naics  711429 non-null  int64  
 5   OCC_CODE                711429 non-null  object 
 6   OCC_TITLE               711429 non-null  object 
 7   emp_occupation          711429 non-null  float64
dtypes: float64(1), int64(3), object(4)
memory usage: 48.8+ MB


In [9]:
ptn_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31204 entries, 0 to 188572
Data columns (total 26 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   State_GEOID   31204 non-null  object
 1   County_GEOID  31204 non-null  object
 2   FIPS          31204 non-null  int64 
 3   naics_2       31204 non-null  object
 4   naics         31204 non-null  object
 5   DESCRIPTION   31204 non-null  object
 6   emp_nf        31204 non-null  object
 7   emp           31204 non-null  int64 
 8   qp1_nf        31204 non-null  object
 9   qp1           31204 non-null  int64 
 10  ap_nf         31204 non-null  object
 11  ap            31204 non-null  int64 
 12  est           31204 non-null  int64 
 13  n<5           31204 non-null  object
 14  n5_9          31204 non-null  object
 15  n10_19        31204 non-null  object
 16  n20_49        31204 non-null  object
 17  n50_99        31204 non-null  object
 18  n100_249      31204 non-null  object
 19  n250_499

## Analysis


In [10]:
def getValueSorted(df, groupBy, sortBy, numResults=20):
    grouped_df = df.groupby(groupBy)[sortBy].sum()
    sorted_df = grouped_df.sort_values(ascending=False)
    sorted_df = sorted_df.reset_index()[groupBy[0]]

    if len(sorted_df) < numResults:
        filled_df = np.full(numResults, None, dtype=object)
        filled_df[: len(sorted_df)] = sorted_df
        return filled_df
    else:
        return sorted_df.head(numResults)

### Top Industries


#### Process


In [11]:
top_industries = pd.DataFrame()

In [12]:
highest_est = ptn_df.groupby(["naics", "DESCRIPTION"])[["emp", "est"]].sum()
highest_est["employment_per_establishment"] = highest_est["emp"] / highest_est["est"]

In [13]:
top_industries["gdp"] = getValueSorted(gdp_df, ["IndustryClassification"], "2022")
top_industries["employment"] = getValueSorted(
    occ_df, ["naics"], "emp_total_county_naics"
)
top_industries["establishments"] = getValueSorted(ptn_df, ["naics"], "est")
top_industries["annual_pay"] = getValueSorted(ptn_df, ["naics"], "ap")
top_industries["employment_per_establishment"] = getValueSorted(
    highest_est, ["naics"], "employment_per_establishment"
)

#### Result


In [14]:
top_industries.head(20)

,gdp,employment,establishments,annual_pay,employment_per_establishment
0,"31-33,51",2382,2382,2382,3361
1,31-33,2381,2383,2211,3311
2,"321,327-339",3261,2381,2362,3364
3,"22,48-49",3330A1,2362,2381,2122
4,"311-316,322-326",3363,3231,2371,3313
5,23,2383,3327,2383,3363
6,"11,21",3364,2371,3330A1,3366
7,22,3320A1,3399,3345,3365
8,21,3320A2,3320A2,3364,3336
9,11,2371,3370A1,3261,3362


**_Durable Goods_** is the most interesting branch, as Pferd-Werkzeuge creates product for this industry in specific. They also seem to have a high gdp, which makes them a better target for premium products.

The **_Construction_** branch could be interesting, as many of the product produces by Pferd, can also be used for woodworking etc. and they have a high number of employments, yet their gdp doesn't seem to follow suit. So we exclude them for now.

Picked NAICS:

1. 3363 Motor Vehicle Parts Manufacturing
2. 3364 Aerospace Product and Parts Manufacturing
3. 3327 Machine Shops; Turned Product; and Screw, Nut, and Bolt Manufacturing
4. 3330A1 Machinery Manufacturing
5. 3320A2 Fabricated Metal Product Manufacturing


In [15]:
top_industries = ["3363", "3364", "3327", "3330A1", "3320A2"]

### Top Occupations


#### Process


In [16]:
top_occupations = pd.DataFrame()


occ_df_filtered_naic1 = occ_df[occ_df["naics"] == top_industries[0]]
occ_df_filtered_rest = occ_df[occ_df["naics"].isin(top_industries[0:])]

In [17]:
top_occupations["naics1"] = getValueSorted(
    occ_df_filtered_naic1, ["OCC_CODE"], "emp_occupation"
)
top_occupations["naics2-5"] = getValueSorted(
    occ_df_filtered_rest, ["OCC_CODE"], "emp_occupation"
)
top_occupations["naics_all"] = getValueSorted(
    occ_df[occ_df["naics"].isin(top_industries)],
    ["OCC_CODE"],
    "emp_occupation",
)

top_occupations["mean"] = (
    occ_df[occ_df["naics"].isin(top_industries)]
    .groupby(["OCC_CODE"])["emp_occupation"]
    .mean()
    .sort_values(ascending=False)
    .reset_index()["OCC_CODE"]
)

#### Results


In [18]:
top_occupations.head(50)

,naics1,naics2-5,naics_all,mean
0,51-4031,51-4041,51-4041,51-2011
1,51-2031,51-4121,51-4121,17-2011
2,17-2112,51-4031,51-4031,49-3011
3,51-4081,17-2112,17-2112,51-4041
4,51-4041,17-2141,17-2141,51-4121
5,49-9071,49-9071,49-9071,17-3021
6,49-9041,49-9041,49-9041,51-4031
7,51-4111,11-3051,11-3051,17-2112
8,51-4121,51-4081,51-4081,51-2031
9,11-3051,51-2031,51-2031,17-2141


Occupations that peak our interest are the ones, that would directly use our products (premium tools for metalworking)

1. 51-4031 Cutting, Punching, and Press Machine Setters, Operators, and Tenders, Metal and Plastic
2. 51-4041 Machinists
3. 51-4121 Welders, Cutters, Solderers, and Brazers Machinists
4. 51-4081 Multiple Machine Tool Setters, Operators, and Tenders, Metal and Plastic
5. 51-2031 Engine and Other Machine Assemblers


In [19]:
top_occupations = ["51-4031", "51-4041", "51-4121", "51-4081", "51-2031"]

## Export


In [20]:
top_picks = pd.DataFrame()

top_picks["naics"] = top_industries
top_picks["occ"] = top_occupations

pd.to_pickle(top_picks, "../data/processed/top_picks.pkl")